# 导入库以及数据集

In [2]:
# 基本工具
import numpy as np
import pandas as pd 
import time
import os #修改环境变量

#算法/损失/评估指标等
import sklearn
from sklearn.ensemble import RandomForestRegressor as RFR # 随机森林回归
from sklearn.model_selection import KFold,cross_validate

# 优化器
from bayes_opt import BayesianOptimization

In [5]:
data = pd.read_csv(r"C:\Users\ZARD\Desktop\超参\程序学习\AutoML与超参数优化——菜菜\train_encode.csv",index_col=0)

# 查看数据集的信息

In [6]:
data.head(5)

,Id,住宅类型,住宅区域,街道接触面积(英尺),住宅面积,街道路面状况,巷子路面状况,住宅形状(大概),住宅现状,水电气,...,泳池面积,泳池质量,篱笆质量,其他配置,其他配置的价值,销售月份,销售年份,销售类型,销售状态,SalePrice
0,0.0,5.0,3.0,36.0,327.0,1.0,0.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,8.0,4.0,208500
1,1.0,0.0,3.0,51.0,498.0,1.0,0.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,1.0,8.0,4.0,181500
2,2.0,5.0,3.0,39.0,702.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,8.0,2.0,8.0,4.0,223500
3,3.0,6.0,3.0,31.0,489.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.0,0.0,140000
4,4.0,5.0,3.0,55.0,925.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,2.0,8.0,4.0,250000


In [4]:
x = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [5]:
x.head()

,Id,住宅类型,住宅区域,街道接触面积(英尺),住宅面积,街道路面状况,巷子路面状况,住宅形状(大概),住宅现状,水电气,...,半开放式门廊面积,泳池面积,泳池质量,篱笆质量,其他配置,其他配置的价值,销售月份,销售年份,销售类型,销售状态
0,0.0,5.0,3.0,36.0,327.0,1.0,0.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,8.0,4.0
1,1.0,0.0,3.0,51.0,498.0,1.0,0.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,8.0,4.0
2,2.0,5.0,3.0,39.0,702.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,8.0,4.0
3,3.0,6.0,3.0,31.0,489.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.0,0.0
4,4.0,5.0,3.0,55.0,925.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,2.0,8.0,4.0


In [7]:
y.head() # 价格

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [6]:
x.shape

(1460, 80)

# 做贝叶斯优化（BayesianOptimization）

## 定义目标函数

In [57]:
# 定义目标函数
def bayesopt_objective(n_estimators,max_depth,max_features,min_impurity_decrease):
    # 定义评估器
    # 需要调整的超参数等于目标函数的输入，不需要调整的超参数则直接等于固定值
    # 默认参数输入一定是浮点数，因此需要套上int函数处理成整数
    reg = RFR(
        n_estimators=int(n_estimators)
        ,max_depth=int(max_depth)
        ,max_features=int(max_features)
        ,min_impurity_decrease=min_impurity_decrease
        ,random_state=1412
        ,verbose=False # 可自行决定是否开启森林建树的verbose
        ,n_jobs=-1
    )
    
    # 定义损失的输出，5折交叉验证下的结果，输出负根均方误差（-RMSE）
    # 注意，交叉验证需要使用数据，但我们不能让数据x,y成为目标函数的输入
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(
        reg,x,y
        ,scoring="neg_root_mean_squared_error" # 负根均方误差
        ,cv=cv
        ,verbose=False # 不需要打印具体流程
        ,n_jobs=-1 
        ,error_score='raise' # 默认值为None，此时返回“空”
        # 如果交叉验证中算法执行报错，则告诉我们错误的理由
    )
    # 交叉验证输出的评估指标是负根均方误差，因此本来就是负的损失
    # 目标函数可直接输出该损失的均值
    return np.mean(validation_loss["test_score"])

## 设置备选参数空间

In [58]:
#备选参数空间
param_grid_simple = {
    'n_estimators':(80,100)
    ,'max_depth':(10,25)
    ,'max_features':(10,20)
    ,'min_impurity_decrease':(0,1)
    }

## 定义优化目标函数的具体流程

In [59]:
# 定义优化目标函数的具体流程
def param_bayes_opt(init_points,n_iter):
    
    # 定义优化器，先实例化优化器
    opt = BayesianOptimization(
          bayesopt_objective # 需要优化的目标函数
        ,param_grid_simple # 备选参数空间
        ,random_state = 1412 # 随机数种子，虽然无法控制
    )
    
    # 使用优化器，其中bayes_opt只支持最大化
    opt.maximize(
        init_points = init_points  # 抽取多少个观测值
        ,n_iter = n_iter # 一共观测/迭代多少次
    )

    #优化完成，取出最佳最佳参数与分数
    params_best = opt.max['params']
    score_best = opt.max["target"]

    #打印最佳参数与最佳分数
    print("\n","\n","best params: ",params_best,
         "\n","\n","best cvscore: ",score_best_best,
         )
    
    # 返回最佳参数与最佳分数
    return params_best,score_best


## 定义验证函数

In [65]:
# 定义验证函数

def bayes_opt_validation(params_best):
    reg = RFR(
            n_estimators = int(params_best["n_estimators"])
            ,max_depth=int(params_best["max_depth"])
            ,min_impurity_decrease = params_best["min_impurity_decrease"]
            ,random_state=1412
            ,verbose=Flase
            ,n_jobs=-1
        )
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(
            reg,x,y
            ,scoring = "neg_root_mean_squared_error"
            ,cv = cv
            ,verbose = False
            ,n_jobs = -1
        )# 执行实际优化流程
    
    return np.mean(validation_loss["test_score"])


## 执行实际优化流程

In [66]:
# 执行实际优化流程
start = time.time()
params_best,score_best = param_bayes_opt(20,280) # 初始看20个观测值，后面迭代280次
print('It take %s minutes'% ((time.time()-start)/60))
validation_score = bayes_opt_validation(params_best)
print("\n","\n","validation_score: ",validation_score)

|   iter    |  target   | max_depth | max_fe... | min_im... | n_esti... |
-------------------------------------------------------------------------
| 1         | -0.471    | 23.2      | 17.52     | 0.06379   | 88.79     |
| 2         | -0.9838   | 14.8      | 17.61     | 0.9214    | 97.58     |
| 3         | -0.6139   | 15.86     | 15.56     | 0.2661    | 87.98     |
| 4         | -0.4759   | 14.05     | 16.84     | 0.06744   | 89.72     |
| 5         | -0.9838   | 18.71     | 19.17     | 0.9315    | 83.7      |
| 6         | -0.9838   | 17.7      | 19.58     | 0.7127    | 89.18     |
| 7         | -0.6474   | 14.21     | 12.62     | 0.3381    | 91.51     |
| 8         | -0.9838   | 23.23     | 10.89     | 0.6078    | 95.06     |
| 9         | -0.9838   | 14.89     | 14.0      | 0.9487    | 80.16     |
| 10        | -0.4528   | 11.52     | 12.58     | 0.03276   | 92.56     |
| 11        | -0.6236   | 13.14     | 13.31     | 0.2563    | 98.24     |
| 12        | -0.6724   | 17.94     | 

| 67        | -0.3543   | 10.43     | 17.19     | 0.0       | 81.91     |
| 68        | -0.4091   | 11.11     | 18.93     | 0.008148  | 82.27     |
| 69        | -0.6104   | 11.98     | 19.05     | 0.2544    | 84.93     |
| 70        | -0.6095   | 10.34     | 17.91     | 0.2817    | 80.06     |
| 71        | -0.9838   | 10.1      | 18.06     | 0.9712    | 83.0      |
| 72        | -0.35     | 12.71     | 15.27     | 0.0       | 85.63     |
| 73        | -0.9838   | 12.47     | 14.5      | 1.0       | 84.71     |
| 74        | -0.3531   | 11.18     | 16.18     | 0.0       | 82.2      |
| 75        | -0.35     | 13.29     | 13.18     | 0.0       | 86.43     |
| 76        | -0.3541   | 11.77     | 17.58     | 0.0       | 81.67     |
| 77        | -0.3496   | 13.36     | 16.48     | 0.0       | 84.88     |
| 78        | -0.4068   | 12.19     | 14.57     | 0.007743  | 82.2      |
| 79        | -0.4556   | 11.69     | 15.95     | 0.03646   | 86.32     |
| 80        | -0.4323   | 13.96     | 

| 134       | -0.4607   | 11.42     | 17.61     | 0.04534   | 83.87     |
| 135       | -0.3547   | 10.12     | 16.3      | 0.0       | 82.78     |
| 136       | -0.3517   | 13.65     | 14.91     | 0.0       | 86.6      |
| 137       | -0.3504   | 17.13     | 15.28     | 0.0       | 85.77     |
| 138       | -0.6239   | 14.53     | 13.03     | 0.2671    | 83.57     |
| 139       | -0.9838   | 10.44     | 13.5      | 0.6662    | 90.13     |
| 140       | -0.3522   | 12.24     | 12.53     | 0.0       | 89.83     |
| 141       | -0.5754   | 12.48     | 13.16     | 0.1174    | 84.03     |
| 142       | -0.3521   | 14.33     | 12.92     | 0.0       | 86.33     |
| 143       | -0.3529   | 12.94     | 19.46     | 0.0       | 90.53     |
| 144       | -0.9838   | 13.64     | 12.72     | 0.6663    | 87.34     |
| 145       | -0.3523   | 13.22     | 12.72     | 0.0       | 89.27     |
| 146       | -0.3531   | 10.0      | 19.53     | 0.0       | 91.5      |
| 147       | -0.3505   | 15.52     | 

| 202       | -0.9838   | 22.98     | 19.91     | 0.708     | 90.17     |
| 203       | -0.3492   | 25.0      | 15.99     | 0.0       | 89.83     |
| 204       | -0.9838   | 25.0      | 10.0      | 0.8217    | 80.0      |
| 205       | -0.35     | 25.0      | 20.0      | 0.0       | 80.0      |
| 206       | -0.3514   | 25.0      | 17.29     | 0.0       | 80.0      |
| 207       | -0.3506   | 22.73     | 18.68     | 0.0       | 80.0      |
| 208       | -0.3511   | 24.99     | 18.81     | 0.0       | 82.34     |
| 209       | -0.3491   | 22.54     | 15.89     | 0.0       | 80.0      |
| 210       | -0.9838   | 23.68     | 16.91     | 0.7643    | 81.33     |
| 211       | -0.3498   | 25.0      | 20.0      | 0.0       | 84.33     |
| 212       | -0.3509   | 20.61     | 14.77     | 0.0       | 80.0      |
| 213       | -0.3489   | 20.89     | 20.0      | 0.0       | 80.0      |
| 214       | -0.3516   | 20.42     | 17.24     | 0.0       | 80.0      |
| 215       | -0.3501   | 18.15     | 

| 269       | -0.3523   | 16.72     | 12.7      | 0.0       | 86.85     |
| 270       | -0.3522   | 11.95     | 15.58     | 0.0       | 85.22     |
| 271       | -0.4661   | 13.07     | 18.93     | 0.05841   | 90.08     |
| 272       | -0.3546   | 10.48     | 16.8      | 0.0       | 83.49     |
| 273       | -0.4724   | 24.74     | 18.81     | 0.06543   | 85.4      |
| 274       | -0.3514   | 12.89     | 11.97     | 0.0       | 86.73     |
| 275       | -0.9838   | 22.47     | 19.82     | 0.6159    | 86.79     |
| 276       | -0.3506   | 15.73     | 15.65     | 0.0       | 86.26     |
| 277       | -0.9838   | 13.4      | 16.5      | 0.7104    | 80.12     |
| 278       | -0.3503   | 15.82     | 12.28     | 0.0       | 86.36     |
| 279       | -0.3542   | 11.72     | 19.1      | 0.0       | 89.57     |
| 280       | -0.8559   | 24.57     | 12.36     | 0.5219    | 89.39     |
| 281       | -0.4081   | 17.99     | 12.81     | 0.00798   | 88.99     |
| 282       | -0.9838   | 13.77     | 

NameError: name 'score_best_best' is not defined

# 对工业蒸汽量预测项目（用其他数据集做优化）

## 导入数据

In [34]:
# 基本工具
import numpy as np
import pandas as pd 
import time
import os #修改环境变量

#算法/损失/评估指标等
import sklearn
from sklearn.ensemble import RandomForestRegressor as RFR # 随机森林回归
from sklearn.model_selection import KFold,cross_validate

# 优化器
from bayes_opt import BayesianOptimization

In [48]:
train_data_file = r"C:\Users\ZARD\Desktop\material\数据集汇总\工业蒸汽量预测\zhengqi_train.txt"
train_data = pd.read_csv(train_data_file, sep='\t', encoding='utf-8')

In [44]:
train_data.head() # 训练集

,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V29,V30,V31,V32,V33,V34,V35,V36,V37,target
0,0.566,0.016,-0.143,0.407,0.452,-0.901,-1.812,-2.360,-0.436,-2.114,...,0.136,0.109,-0.615,0.327,-4.627,-4.789,-5.101,-2.608,-3.508,0.175
1,0.968,0.437,0.066,0.566,0.194,-0.893,-1.566,-2.360,0.332,-2.114,...,-0.128,0.124,0.032,0.600,-0.843,0.160,0.364,-0.335,-0.730,0.676
2,1.013,0.568,0.235,0.370,0.112,-0.797,-1.367,-2.360,0.396,-2.114,...,-0.009,0.361,0.277,-0.116,-0.843,0.160,0.364,0.765,-0.589,0.633
3,0.733,0.368,0.283,0.165,0.599,-0.679,-1.200,-2.086,0.403,-2.114,...,0.015,0.417,0.279,0.603,-0.843,-0.065,0.364,0.333,-0.112,0.206
4,0.684,0.638,0.260,0.209,0.337,-0.454,-1.073,-2.086,0.314,-2.114,...,0.183,1.078,0.328,0.418,-0.843,-0.215,0.364,-0.280,-0.028,0.384


In [45]:
x = train_data.iloc[:,:-1]
y = train_data.iloc[:,-1]

In [50]:
x.shape

(2888, 38)

## 做贝叶斯优化

### 定义目标函数

In [51]:
# 定义目标函数
def bayesopt_objective(n_estimators,max_depth,max_features,min_impurity_decrease):
    # 定义评估器
    # 需要调整的超参数等于目标函数的输入，不需要调整的超参数则直接等于固定值
    # 默认参数输入一定是浮点数，因此需要套上int函数处理成整数
    reg = RFR(
        n_estimators=int(n_estimators)
        ,max_depth=int(max_depth)
        ,max_features=int(max_features)
        ,min_impurity_decrease=min_impurity_decrease
        ,random_state=1412
        ,verbose=False # 可自行决定是否开启森林建树的verbose
        ,n_jobs=-1
    )
    
    # 定义损失的输出，5折交叉验证下的结果，输出负根均方误差（-RMSE）
    # 注意，交叉验证需要使用数据，但我们不能让数据x,y成为目标函数的输入
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(
        reg,x,y
        ,scoring="neg_root_mean_squared_error" # 负根均方误差
        ,cv=cv
        ,verbose=False # 不需要打印具体流程
        ,n_jobs=-1 
        ,error_score='raise' # 默认值为None，此时返回“空”
        # 如果交叉验证中算法执行报错，则告诉我们错误的理由
    )
    # 交叉验证输出的评估指标是负根均方误差，因此本来就是负的损失
    # 目标函数可直接输出该损失的均值
    return np.mean(validation_loss["test_score"])

### 参数空间

In [52]:
#备选参数空间
param_grid_simple = {
    'n_estimators':(80,100)
    ,'max_depth':(10,25)
    ,'max_features':(10,20)
    ,'min_impurity_decrease':(0,1)
    }

### 优化的具体流程

In [53]:
# 定义优化目标函数的具体流程
def param_bayes_opt(init_points,n_iter):
    
    # 定义优化器，先实例化优化器
    opt = BayesianOptimization(
          bayesopt_objective # 需要优化的目标函数
        ,param_grid_simple # 备选参数空间
        ,random_state = 1412 # 随机数种子，虽然无法控制
    )
    
    # 使用优化器，其中bayes_opt只支持最大化
    opt.maximize(
        init_points = init_points  # 抽取多少个观测值
        ,n_iter = n_iter # 一共观测/迭代多少次
    )

    #优化完成，取出最佳最佳参数与分数
    params_best = opt.max['params']
    score_best = opt.max["target"]

    #打印最佳参数与最佳分数
    print("\n","\n","best params: ",params_best,
         "\n","\n","best cvscore: ",score_best_best,
         )
    
    # 返回最佳参数与最佳分数
    return params_best,score_best

### 验证函数

In [67]:
# 定义验证函数

def bayes_opt_validation(params_best):
    reg = RFR(
            n_estimators = int(params_best["n_estimators"])
            ,max_depth=int(params_best["max_depth"])
            ,min_impurity_decrease = params_best["min_impurity_decrease"]
            ,random_state=1412
            ,verbose=Flase
            ,n_jobs=-1
        )
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(
            reg,x,y
            ,scoring = "neg_root_mean_squared_error"
            ,cv = cv
            ,verbose = False
            ,n_jobs = -1
        )# 执行实际优化流程
    
    return np.mean(validation_loss["test_score"])

### 执行优化流程

In [68]:
# 执行实际优化流程
start = time.time()
params_best,score_best = param_bayes_opt(20,280) # 初始看20个观测值，后面迭代280次
print('It take %s minutes'% ((time.time()-start)/60))
validation_score = bayes_opt_validation(params_best)
print("\n","\n","validation_score: ",validation_score)

|   iter    |  target   | max_depth | max_fe... | min_im... | n_esti... |
-------------------------------------------------------------------------
| 1         | -0.471    | 23.2      | 17.52     | 0.06379   | 88.79     |
| 2         | -0.9838   | 14.8      | 17.61     | 0.9214    | 97.58     |
| 3         | -0.6139   | 15.86     | 15.56     | 0.2661    | 87.98     |
| 4         | -0.4759   | 14.05     | 16.84     | 0.06744   | 89.72     |
| 5         | -0.9838   | 18.71     | 19.17     | 0.9315    | 83.7      |
| 6         | -0.9838   | 17.7      | 19.58     | 0.7127    | 89.18     |
| 7         | -0.6474   | 14.21     | 12.62     | 0.3381    | 91.51     |
| 8         | -0.9838   | 23.23     | 10.89     | 0.6078    | 95.06     |
| 9         | -0.9838   | 14.89     | 14.0      | 0.9487    | 80.16     |
| 10        | -0.4528   | 11.52     | 12.58     | 0.03276   | 92.56     |
| 11        | -0.6236   | 13.14     | 13.31     | 0.2563    | 98.24     |
| 12        | -0.6724   | 17.94     | 

| 67        | -0.3503   | 18.23     | 13.3      | 0.0       | 98.16     |
| 68        | -0.6445   | 18.44     | 14.53     | 0.3962    | 99.76     |
| 69        | -0.9838   | 18.8      | 10.06     | 0.8565    | 97.73     |
| 70        | -0.352    | 16.56     | 10.92     | 0.0       | 96.57     |
| 71        | -0.3498   | 18.32     | 11.99     | 0.0       | 97.91     |
| 72        | -0.435    | 18.61     | 10.66     | 0.02094   | 99.21     |
| 73        | -0.3502   | 19.61     | 13.18     | 0.0       | 97.56     |
| 74        | -0.9838   | 18.44     | 10.0      | 0.9509    | 92.56     |
| 75        | -0.6509   | 15.32     | 10.05     | 0.2492    | 91.96     |
| 76        | -0.632    | 19.09     | 14.47     | 0.3441    | 96.77     |
| 77        | -0.9838   | 19.1      | 13.19     | 0.7771    | 98.38     |
| 78        | -0.3498   | 19.48     | 11.99     | 0.0       | 97.3      |
| 79        | -0.3509   | 17.46     | 12.98     | 0.0       | 97.31     |
| 80        | -0.9838   | 11.81     | 

| 135       | -0.9838   | 15.47     | 14.48     | 1.0       | 85.53     |
| 136       | -0.35     | 12.92     | 15.47     | 0.0       | 85.27     |
| 137       | -0.3506   | 15.44     | 15.76     | 0.0       | 83.46     |
| 138       | -0.9838   | 12.08     | 16.69     | 0.6236    | 84.7      |
| 139       | -0.3513   | 15.59     | 16.95     | 0.0       | 84.81     |
| 140       | -0.3521   | 14.15     | 13.16     | 0.0       | 82.95     |
| 141       | -0.3499   | 17.14     | 13.61     | 0.0       | 93.88     |
| 142       | -0.3527   | 12.18     | 10.42     | 0.0       | 82.66     |
| 143       | -0.3549   | 10.63     | 10.0      | 0.0       | 83.58     |
| 144       | -0.766    | 10.27     | 10.7      | 0.4593    | 82.13     |
| 145       | -0.4609   | 10.17     | 10.03     | 0.03842   | 84.92     |
| 146       | -0.3517   | 18.11     | 14.97     | 0.0       | 93.67     |
| 147       | -0.4645   | 18.72     | 13.53     | 0.04622   | 92.53     |
| 148       | -0.3506   | 17.03     | 

| 203       | -0.6483   | 16.89     | 16.02     | 0.4485    | 83.36     |
| 204       | -0.3518   | 14.29     | 12.45     | 0.0       | 96.05     |
| 205       | -0.3506   | 16.82     | 11.69     | 0.0       | 92.23     |
| 206       | -0.3515   | 14.45     | 10.7      | 0.0       | 90.09     |
| 207       | -0.3504   | 14.26     | 16.17     | 0.0       | 82.94     |
| 208       | -0.351    | 20.03     | 13.83     | 0.0       | 93.42     |
| 209       | -0.4182   | 16.93     | 12.89     | 0.01197   | 90.99     |
| 210       | -0.3526   | 12.84     | 12.55     | 0.0       | 81.4      |
| 211       | -0.9838   | 13.38     | 12.27     | 0.9832    | 81.95     |
| 212       | -0.3502   | 16.0      | 12.63     | 0.0       | 92.93     |
| 213       | -0.3504   | 21.02     | 13.44     | 0.0       | 97.39     |
| 214       | -0.9838   | 16.58     | 12.47     | 1.0       | 92.18     |
| 215       | -0.3513   | 15.24     | 16.91     | 0.0       | 82.53     |
| 216       | -0.3504   | 14.1      | 

| 271       | -0.3512   | 22.16     | 16.7      | 0.0       | 100.0     |
| 272       | -0.9838   | 23.04     | 13.12     | 0.6319    | 80.02     |
| 273       | -0.9838   | 24.07     | 16.6      | 0.5984    | 90.73     |
| 274       | -0.4844   | 22.79     | 17.8      | 0.07425   | 99.71     |
| 275       | -0.5511   | 11.72     | 11.29     | 0.1003    | 83.04     |
| 276       | -0.3514   | 16.76     | 18.78     | 0.0       | 85.06     |
| 277       | -0.3515   | 23.6      | 10.92     | 0.0       | 80.0      |
| 278       | -0.3508   | 15.65     | 11.68     | 0.0       | 96.21     |
| 279       | -0.9838   | 16.1      | 18.92     | 0.8463    | 84.81     |
| 280       | -0.3524   | 12.87     | 10.0      | 0.0       | 94.06     |
| 281       | -0.3501   | 13.67     | 13.18     | 0.0       | 87.37     |
| 282       | -0.3867   | 15.68     | 17.4      | 0.003046  | 83.65     |
| 283       | -0.3502   | 24.81     | 11.41     | 0.0       | 80.0      |
| 284       | -0.3501   | 16.04     | 

NameError: name 'score_best_best' is not defined